In [3]:
from gensim.models import KeyedVectors

In [179]:
file = '100000-small.txt'

In [180]:
wv_from_text = KeyedVectors.load_word2vec_format(file,binary=False)

In [181]:
wv_from_text.init_sims(replace=True)

In [14]:
xinwen_list = []
with open('xinwen.txt','r',encoding='utf-8') as f:
    for line in f:
        xinwen_list.append(line)

In [337]:
for i in range(len(xinwen_list)):
    xinwen_list[i] = xinwen_list[i].replace('\n','')

In [338]:
first_news = xinwen_list[1]

In [340]:
first_news

'人民网东京4月2日电(吴颖)日本滋贺县草津市1日公布的消息称，该市于3月31日收到来自中国友好城市上海市徐汇区捐赠的1万只口罩，后续将通过草津栗东医师会向草津市和栗东市内的医疗机构分发。据该市透露，上海市徐汇区政府3月23日提出捐赠意愿后，通过空运于31日寄达。全部物资由草津栗东医师会接收。该医师会事务局长松村元喜表示：“听说现在日本的口罩供应商已不接订单，很多医护人员已经没有口罩库存。非常感谢徐汇区的捐赠。计划优先口罩紧缺的诊所、医院陆续分发。”据悉，截至3月24日，与日本滋贺县建立友好关系的湖南省累计向该县捐赠了2万只医用口罩物资。滋贺县计划近期分发给口罩紧缺的儿童福利设施和残疾人支援设施等。截至4月2日的数据显示,日本滋贺县累计报告新冠肺炎病例7例。滋贺县知事三日月大造呼吁县民，尽量避免前往疫情不断扩大的京都府、大阪府、爱知县等9个临近府县。'

In [353]:
stop_words = []
with open('stopwords.txt','r',encoding='utf-8') as f:
    for i in f:
        stop_words.append(i.replace('\n',''))

In [339]:
import jieba

In [371]:
def cut_dict(first_news):    
    first_news_1 = []
    for i in first_news:
        if i not in stop_words:
            first_news_1.append(i)
    w_dict = {}
    for i,word in enumerate(first_news_1):
        window_list = []
        for j in range(3):
            if (i+(j+1)) < len(first_news_1):
                window_list.append(first_news_1[i+(j+1)])
            if (i-(j+1)) > -1:
                window_list.append(first_news_1[i-(j+1)])
        if word in w_dict:
            w_dict[word] += window_list
        else:
            w_dict[word] = window_list 
    for word in w_dict:
        w_dict[word] = [i for i in set(w_dict[word])]
    w_dict_weight = {}
    for i in w_dict:
        w_dict_weight[i] = 1
    return w_dict,w_dict_weight

In [358]:
d = 0.85

In [363]:
def sum_V_jk(V_j):
    sum_v_jk = 0
    if V_j in wv_from_text:
        for V_k in w_dict[V_j]:
            if V_k in wv_from_text:
                sum_v_jk += wv_from_text.similarity(V_j,V_k)
            else:
                sum_v_jk += 0.45
    else:
        sum_v_jk = 0.45*len(w_dict[V_j])
    return sum_v_jk

In [364]:
def sum_V_ij(V_i):
    sum_v_ij = 0
    if V_i in wv_from_text:
        for V_j in w_dict[V_i]:
            sum_v_jk = sum_V_jk(V_j) 
            if V_j in wv_from_text:
                sum_v_ij = (wv_from_text.similarity(V_i,V_j) * w_dict_weight[V_j] / sum_v_jk) + sum_v_ij
            else:
                sum_v_ij = (0.45 / sum_v_jk) + sum_v_ij
    else:
        for V_j in w_dict[V_i]:
            sum_v_jk = sum_V_jk(V_j)
            sum_v_ij = (0.45/sum_v_jk) + sum_v_ij
    return sum_v_ij

In [365]:
def WS(V_i):
    ws_weight = (1-d) + (d * sum_V_ij(V_i))
    w_dict_weight[V_i] = ws_weight

In [393]:
def num_it():
    for i in range(5):
        for word in w_dict:
            WS(word)

In [416]:
first_news = jieba.lcut(xinwen_list[4])

In [417]:
w_dict, w_dict_weight = cut_dict(first_news)

In [418]:
num_it()

In [421]:
[i[0] for i in sorted(w_dict_weight.items(),key=lambda x:x[1],reverse=True)][:30]

['中国',
 '疫情',
 '美国',
 '物资',
 '抗疫',
 '全球',
 '提供',
 '国家',
 '医疗',
 '援助',
 '西方',
 '国际',
 '做',
 '口罩',
 '新冠',
 '病毒',
 '寻求',
 '动机',
 '贡献',
 '世界',
 '污蔑',
 '感谢',
 '危机',
 '组织',
 '欧盟',
 '模式',
 '超越',
 '贝格',
 '德黑兰',
 '斯托尔']

In [420]:
xinwen_list[4]

'载着中国医生和医疗物资的飞机抵达罗马、德黑兰等世界各地的机场，这些照片在全球社交媒体上广泛流传。 不少国家在寻求中国的帮助。欧盟减少医疗物资出口之后，塞尔维亚总统求助中国提供必需的物资，几天后，欧盟委员会主席也公开感谢中国的贡献……“投桃报李”“患难相扶”是中国传统处世哲学。在中国疫情吃紧时，欧盟曾援助中国数十吨医疗物资。在全人类共同的危机面前，中国正在做的是：回报善意，尽最大努力去帮助别人。但在个别西方政客的眼里，这却成为近期新冠疫情全球大暴发，欧洲、美国相继成为“震中”后，他们深以为忧虑的画面。他们担忧的不是疫情下公众的生命安全，而是中国的国际形象可能因此提升甚至超越美国。30日，美国国务卿蓬佩奥同北约秘书长斯托尔滕贝格通话，谈及“反击”中国传播与新冠病毒有关虚假信息和宣传活动的重要性。但哪些信息是虚假的？是中方的抗疫成效？还是中方向其他国家提供的支持和帮助？美方却无法明确指出。妄图借污蔑“中国病毒”洗白自身抗疫不力后，某些西方国家又恶意揣测中国的援助动机。他们不愿承认的是，病毒不分种族、国界和意识形态，人的生命和安全才是第一。他们在意的是：中国模式永远不能超越西方。假如中国实在无可辩驳地做得更好，那么必须选择视而不见，或者颠倒黑白，污蔑中国“动机不纯”。世界报业辛迪加网站3月30日发表文章指出，疫情大流行正在全球展开，美国却在考虑如何在与中国交手中占上风。不过美国胜利的希望日渐渺茫。对于世界上大多数人来说，重要的不是疫情从哪里开始，而是如何结束。在帮助结束疫情方面，中国比美国做得要多得多。文章强调，美国不能阻止全世界仿效中国在应对疫情危机中的方法和组织模式。并且到目前为止，中国的抗疫记录十分令人信服。相比之下，“没有人寻求过美国的任何帮助，更不要说感谢了。美国也没有提供过任何帮助。”在全球亟需团结合作，共同抗击疫情的时候，鼓噪新一轮抹黑中国，这跟当前国际社会团结抗疫的气氛十分不和谐。对于那些指责中国进行“口罩外交”的人，正如中国外交部发言人华春莹所说：“你们真的想让中国对于在疫情中挣扎的人们坐视不理吗？”当前，中国政府已向120个国家和4个国际组织提供了包括普通医用口罩、N95口罩、防护服、核酸检测试剂、呼吸机等在内的物资援助。中国的企业正在日以继夜赶制医疗物资，支援国内国际的疫情防控。中国人民的担当和贡献理应受到尊敬而不是诋毁。'